In [24]:
import pennylane as qml
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,f1_score
import pandas as pd

In [25]:
# Load the data credit card dataset
data = pd.read_csv('creditcard.csv')
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [26]:
#take 800 class=0 data and join it with all class =1 data and shuffle the data
data0 = data[data['Class'] == 0].sample(800)
data0
#take 492 class=1 data
data1 = data[data['Class'] == 1]
#data0+data1
data = pd.concat([data0, data1])
data        
#shuffle randomly 'data dataframe'
data = data.sample(frac=1).reset_index(drop=True)
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,153875.0,-0.613696,3.698772,-5.534941,5.620486,1.649263,-2.335145,-0.907188,0.706362,-3.747646,...,0.319261,-0.471379,-0.075890,-0.667909,-0.642848,0.070600,0.488410,0.292345,0.00,1
1,136928.0,-0.750785,1.143111,0.538773,2.596361,1.926681,-0.840333,1.227238,-0.307234,-1.930735,...,0.309680,0.955318,-0.553479,0.105984,0.582849,0.175552,-0.077562,-0.025409,7.87,0
2,119344.0,2.071272,-0.027110,-1.118870,0.378534,-0.036201,-1.119438,0.210676,-0.349191,0.489209,...,-0.293692,-0.693019,0.316249,-0.099055,-0.265096,0.203609,-0.068405,-0.060393,2.69,0
3,154599.0,0.667714,3.041502,-5.845112,5.967587,0.213863,-1.462923,-2.688761,0.677764,-3.447596,...,0.329760,-0.941383,-0.006075,-0.958925,0.239298,-0.067356,0.821048,0.426175,6.74,1
4,15817.0,-4.641893,2.902086,-1.572939,2.507299,-0.871783,-1.040903,-1.593901,-3.254905,1.908963,...,1.963597,-0.217414,-0.549340,0.645545,-0.354558,-0.611764,-3.908080,-0.671248,11.39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,91365.0,-0.847192,0.791081,2.295948,3.253385,0.022321,1.345765,0.919012,0.177825,0.118820,...,-0.154044,-0.389511,0.529980,0.975211,-0.527899,-0.440064,0.091361,0.201849,195.66,0
1288,74980.0,1.112366,-0.395802,1.062520,0.638890,-0.835962,0.641633,-0.921523,0.473407,0.940082,...,-0.051521,-0.040757,0.025694,-0.304825,0.138461,0.354274,0.017817,0.007310,11.50,0
1289,160791.0,2.132386,0.705608,-3.530759,0.514779,1.527175,-1.716268,1.132791,-0.574214,0.128904,...,0.163739,0.703910,-0.245076,0.460049,0.920281,-0.216586,-0.026219,-0.025001,1.00,1
1290,72170.0,1.288976,-0.479226,-0.044827,-0.755642,-0.614088,-0.724636,-0.098219,-0.208441,-1.445252,...,-0.737376,-1.838289,0.184748,-0.002240,0.002286,0.619075,-0.085880,0.005508,60.99,0


In [27]:
# Extract features and labels
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values


In [28]:
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [29]:
# Apply PCA to reduce the number of features to 2
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)


In [30]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [31]:
# Define the quantum device
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)


In [32]:
# Quantum feature map
@qml.qnode(dev)
def quantum_feature_map(params, x):
    qml.templates.StronglyEntanglingLayers(params, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [33]:
# Initialize the quantum circuit parameters
params = np.random.uniform(low=0, high=2 * np.pi, size=(3, n_qubits,3))

In [34]:
# Apply the quantum feature map to the training and testing data
feature_map_train = np.array([quantum_feature_map(params, x) for x in X_train])
feature_map_test = np.array([quantum_feature_map(params, x) for x in X_test])


In [35]:
# Train a classical SVM on the quantum features
svm_classifier = SVC()
svm_classifier.fit(feature_map_train, y_train)

SVC()

In [36]:
# Predict on the testing data
y_pred = svm_classifier.predict(feature_map_test)

In [39]:
# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Classification Accuracy: {accuracy}")

Classification Accuracy: 0.6254826254826255
